In [1]:
!pip install angr capstone keystone-engine

In [2]:
%load_ext autoreload
%autoreload 2
from unicorn import *
from unicorn.arm64_const import *
import unicorn.arm_const
import struct
import capstone
import keystone

In [3]:

disassembler = capstone.Cs(capstone.CS_ARCH_ARM64,capstone.CS_MODE_ARM)
def disas(code, addr):
    for insn in disassembler.disasm(code, addr):
        print("0x%x:\t%s\t%s" % (insn.address, insn.mnemonic, insn.op_str))
        
def gen_shellcode(data,address):
    ks = keystone.Ks(keystone.KS_ARCH_ARM64,keystone.KS_MODE_LITTLE_ENDIAN)
    ret=ks.asm(data,address)
    return bytes(ret[0])

In [4]:
#MAP file to offset
with open("abl_210817","rb") as f:
    data=f.read()

#some datas
fastboot_read = 0xFFFF0000F8871408
download_buffer = 0xffff000090700000
__debug_stdio_write = 0xFFFF0000F88A6D5C
fastboot_write = 0xFFFF0000F8871484
pixel_loader_entry_run = 0xFFFF0000F8813D74
stop_fastboot = 0xFFFF0000F8ACBD20

ABL_LOAD_ADDRESS = 0xFFFF0000F8800000
MEMORY_START = 0xFFFF0000F8000000
MEMORY_SIZE = 200*1024*1024
STACK_START = MEMORY_START + MEMORY_SIZE - 0x1000

def print_stack(uc,num=20):
    sp = uc.reg_read(UC_ARM64_REG_SP)
    print(f"## STACK DUMP ##\nSP: {sp:x}")
    for i,s in enumerate(range(sp,sp+num*8,8)):
        if sp != 0:
            v = struct.unpack("Q",uc.mem_read(s,8))[0]
            print(f"@{s:x} {v:x} - #{i*8:x}")
    
def print_regs(uc):
    for reg in ["X0","X1","X2","X3","X8","X19","X20","X21","X22","X23","X24","X28","X29","X30","SP","PC"]:
        val = eval(f"uc.reg_read(UC_ARM64_REG_{reg})")
        print(f"{reg} - {val:8X}",end=" ")
    print("")

# callback for tracing basic blocks
def hook_block(uc, address, size, user_data):
    print(">>> Tracing basic block at 0x%x, block size = 0x%x" %(address, size))

# callback for tracing instructions
def hook_code(uc, address, size, user_data):
    print("\n############################################")
    print(f"@{address:x} step size {size:x}")
    disas(uc.mem_read(address,size),address)
    print_regs(uc)
    print_stack(uc)
    
def hook_mrs(uc, reg, cp_reg, _):
    print(f">>> Hook MRS instruction: reg = 0x{reg:x}(UC_ARM64_REG_X2) cp_reg = {cp_reg}")
    uc.reg_write(reg, 0x114514)
    print(">>> Write 0x114514 to X")

    # Skip MRS instruction
    return True

def hook_smc(uc, reg, cp_reg, _):
    print(f">>> Hook SMC instruction: reg = 0x{reg:x}(UC_ARM64_REG_X2) cp_reg = {cp_reg}")
    #uc.reg_write(reg, 0x114514)

    # Skip MRS instruction
    return True

def hook_intr(uc, intno, user_data):
    print("\n############################################")
    #only handle Linux syscall
    
    #print_regs(uc)
    #print_stack(uc)

    pc = uc.reg_read(UC_ARM64_REG_PC)
    print(f"got interrupt {intno:02x} {pc:02x}");
    uc.reg_write(UC_ARM64_REG_PC,pc+4)
    uc.emu_stop()
    return False

def hook_mem_invalid(uc,uc_mem_type,addr,size,value,user_data):
    print("\n############################################")
    pc = uc.reg_read(UC_ARM64_REG_PC)
    print(f"INVALID MEMORY @{pc:08X} {addr:08x} {size:08x} {value:08x}")
    print_regs(uc)
    print_stack(uc)
    print("############################################\n")
    return False

#Auto allocate pages of memory of size 10Mega on invalid memory access
PAGE_SIZE=10*1024*1024
def hook_mem_invalid_auto(uc,uc_mem_type,addr,size,value,user_data):
    pc = uc.reg_read(UC_ARM64_REG_PC)
    start = addr & ~(PAGE_SIZE-1)
    print(f"~~~~~~~~~~~~~~                      mu.mem_map(0x{start:08x}, PAGE_SIZE)")
    mu.mem_map(start,PAGE_SIZE)
    return True
    
def hook_fastboot_read(uc,address,size,user_data):
    dest = mu.reg_read(UC_ARM64_REG_X0)
    size = mu.reg_read(UC_ARM64_REG_X1)
    num_read = mu.reg_read(UC_ARM64_REG_X2)

    #print_stack(uc)
    #If we change PC after calling emu_stop() that won't work!
    global commands
    if len(commands):
        command = commands.pop(0)
    else:
        command = None
        
    if command == b"outofloop" or not command:
        #Once we are done, set stop_fastboot to 0 and log instructions
        uc.mem_write(stop_fastboot,struct.pack("Q",1))
        mu.reg_write(UC_ARM64_REG_X0,0xFFFFFFF0)
        uc.reg_write(UC_ARM64_REG_PC,0xFFFF0000F887143C)
    else:
        print(">>> fastboot_read",hex(dest),hex(size),hex(num_read))
        #print_stack(uc)
        mu.mem_write(num_read,struct.pack("Q",len(command)))
        mu.mem_write(dest,command)
        mu.reg_write(UC_ARM64_REG_X0,0)
        uc.reg_write(UC_ARM64_REG_PC,0xFFFF0000F887143C)
    
def hook_fprintf_output(uc,address,size,user_data):
    data = mu.reg_read(UC_ARM64_REG_X0)
    s = mu.mem_read(data,50)
    size = mu.reg_read(UC_ARM64_REG_X1)
    dunno = mu.reg_read(UC_ARM64_REG_X2)
    d = mu.mem_read(dunno,50)
    print(">>> hook_fprintf_output",hex(data),s,hex(size),hex(dunno),d)
    
def hook_stdio_write(uc,address,size,user_data):
    a = mu.reg_read(UC_ARM64_REG_X0)
    b = mu.reg_read(UC_ARM64_REG_X1)
    c = mu.reg_read(UC_ARM64_REG_X2)
    b = mu.mem_read(b,c)
    try:
        b=b.decode("utf-8")
    except Exception as e:
        b=""
    print(b,end="")
    
def hook_fastboot_write(uc,address,size,user_data):
    a = mu.reg_read(UC_ARM64_REG_X0)
    b = mu.reg_read(UC_ARM64_REG_X1)
    
    try:
        s = mu.mem_read(a,b).decode("utf-8")
    except Exception as e:
        s=""
    print("<<<",s)

In [5]:
commands=[
          b"flashing unlock",
          b"oem dmesg",
          b"outofloop",
         ]
              

try:
    # Initialize emulator in ARM mode
    mu = Uc(UC_ARCH_ARM64, UC_MODE_ARM)

    mu.mem_map(MEMORY_START, MEMORY_SIZE)
    mu.mem_map(0xd8000000, PAGE_SIZE)
    mu.mem_map(0xf8200000, PAGE_SIZE)
    mu.mem_map(0xffffffff19200000, PAGE_SIZE)
    mu.mem_map(0xfffffffff8200000, PAGE_SIZE)
    mu.mem_map(0xffff000080000000, PAGE_SIZE)
    mu.mem_map(0xffff000002000000, PAGE_SIZE)
    mu.mem_map(0xffffffff10000000, PAGE_SIZE)
    mu.mem_map(download_buffer,1024*1024*5) #download buffer
    
    #Init SIMD
    SIMD_INIT=gen_shellcode("mov x1, #(0x3 << 20);msr cpacr_el1, x1;isb;STP Q1, Q2, [SP,#0x10]",download_buffer)
    mu.mem_write(download_buffer,SIMD_INIT)
    mu.emu_start(download_buffer, 0,count=3)    

    # write machine code to be emulated to memory
    mu.mem_write(ABL_LOAD_ADDRESS, data)    

    # tracing all basic blocks with customized callback
    #mu.hook_add(UC_HOOK_BLOCK, hook_block)

    # tracing all instruction with customized callback
    #mu.hook_add(UC_HOOK_CODE, hook_code)
    
    mu.hook_add(UC_HOOK_INTR, hook_intr)
    
    if True:
        mu.hook_add(UC_HOOK_MEM_INVALID, hook_mem_invalid_auto)
    else:
        mu.hook_add(UC_HOOK_MEM_INVALID, hook_mem_invalid)
    
    mu.hook_add(UC_HOOK_CODE, hook_fastboot_read, begin=fastboot_read,end=fastboot_read)
    mu.hook_add(UC_HOOK_CODE, hook_stdio_write, begin=__debug_stdio_write,end=__debug_stdio_write)
    mu.hook_add(UC_HOOK_CODE, hook_fastboot_write, begin=fastboot_write,end=fastboot_write)
    
    # Hook MRS instruction
    #mu.hook_add(UC_HOOK_INSN, hook_mrs, None, 1, 0, UC_ARM64_INS_MRS)
    
    # Hook SMC instruction
    #mu.hook_add(UC_HOOK_INSN, hook_smc, None, 1, 0, U)

    # emulate machine code in infinite time
    mu.reg_write(UC_ARM64_REG_SP,STACK_START)
    
    mu.emu_start(pixel_loader_entry_run, 0,count=40000)
        
    pc = mu.reg_read(UC_ARM64_REG_PC)    
    print(f">>> PC = 0x{pc:x}")
    
except UcError as e:
    print("ERROR: %s" % e),

>>> fastboot_read 0xffff0001047fef7c 0x40 0xffff0001047fef70
[   0.000000] [I] [FB] Accept cmd:flashing unlock
<<< INFOdevice already unlocked
<<< OKAY
>>> fastboot_read 0xffff0001047fef7c 0x40 0xffff0001047fef70
[   0.000000] [I] [FB] Accept cmd:oem dmesg
<<< OKAY
panic (caller 0xffff0000f886e798): mutex_acquire_timeout: thread 0x0 (~~~~~~~~~~~~~~                      mu.mem_map(0x00000000, PAGE_SIZE)
) tried to acquire mutex 0xffff0000f8992d70 it already owns.
[   0.000000] [I] [GS] halt action=0, reason=10
[   0.000000] [C] [GS] Rebooting in 5 seconds, press 'c' to cancel: 5>>> PC = 0xffff0000f8819390
